In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, cohen_kappa_score
)
import time
import pennylane as qml
from tqdm import tqdm  # For progress tracking

# ==================================================
# Load and Preprocess Data
# ==================================================

# Load dataset
data = pd.read_csv("data-15s.csv")

# Separate features and target
X = data.iloc[:, :-1]  # Features (first 23 columns)
y = data.iloc[:, -1]   # Target (24th column)

# Encode target labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Convert "b'Non-VPN'" and "b'VPN'" to 0 and 1

# Normalize features (without replacing -1)
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# ==================================================
# Basis Embedding
# ==================================================

def basis_encoding(data, n_qubits):
    """Encode normalized data into binary representation."""
    binary_str = ''.join(format(int(x * (2**n_qubits - 1)), f'0{n_qubits}b') for x in data)
    return binary_str

def apply_basis_embedding(X, n_qubits):
    """Apply basis embedding to the dataset."""
    # Initialize the device with the correct number of qubits
    dev = qml.device("default.qubit", wires=n_qubits)
    
    @qml.qnode(dev)
    def basis_circuit(binary_str):
        """Apply basis encoding to the quantum circuit."""
        for wire, bit in enumerate(binary_str):
            if bit == '1':
                qml.PauliX(wires=wire)
        return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]
    
    encoded_data = []
    for x in tqdm(X, desc="Applying Basis Encoding", unit="sample"):
        binary_str = basis_encoding(x, n_qubits)
        encoded_data.append(basis_circuit(binary_str))
    return np.array(encoded_data)

# ==================================================
# Train and Evaluate Models
# ==================================================

def evaluate_model(model, X_train, X_test, y_train, y_test):
    """Train and evaluate a model."""
    start_time = time.time()
    model.fit(X_train, y_train)
    fit_time = time.time() - start_time

    start_time = time.time()
    y_pred = model.predict(X_test)
    predict_time = time.time() - start_time

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_test, y_pred)

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1": f1,
        "ROC AUC": roc_auc,
        "Kappa": kappa,
        "Fit Time": fit_time,
        "Predict Time": predict_time,
    }

# Define models
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, eval_metric="logloss"),  # Removed use_label_encoder
}



In [15]:
# Test for PCA components = 2, 5, 8
pca_components = [2]
results = {}

for n_components in pca_components:
    print(f"\nTesting with PCA components = {n_components}...")
    
    # Apply PCA
    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X_normalized)
    
    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.3, random_state=42)
    
    # Apply basis embedding
    n_qubits = n_components  # Number of qubits = number of PCA components
    X_train_embedded = apply_basis_embedding(X_train, n_qubits)
    X_test_embedded = apply_basis_embedding(X_test, n_qubits)
    
    # Train and evaluate models
    for model_name, model in models.items():
        print(f"\nTraining {model_name} with PCA components = {n_components}...")
        results[f"{model_name} (PCA={n_components})"] = evaluate_model(model, X_train_embedded, X_test_embedded, y_train, y_test)

# Display results
results_df = pd.DataFrame(results).T
print(results_df)


Testing with PCA components = 2...


Applying Basis Encoding:   0%|                                                           | 0/13130 [00:00<?, ?sample/s]


WireError: Cannot run circuit(s) on default.qubit as they contain wires not found on the device: {4}

In [16]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
from tabulate import tabulate
import time

# Load and prepare data
def prepare_data(filepath):
    df = pd.read_csv(filepath)
    X = df.iloc[:, :23]
    y = df.iloc[:, 23]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_scaled = scaler.fit_transform(X)
    
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    return X_scaled, y_encoded

# Quantum circuit definition
def create_quantum_circuit(n_qubits, n_features):
    dev = qml.device("default.qubit", wires=n_qubits)
    
    @qml.qnode(dev)
    def circuit(inputs):
        for i in range(n_features):
            qml.RX(inputs[i], wires=i % n_qubits)
        
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])
            
        return [qml.expval(qml.PauliZ(i)) for i in range(2)]
    
    return circuit

# Quantum embedding
def quantum_embedding(X_scaled):
    n_qubits = 4
    n_samples = X_scaled.shape[0]
    n_features = X_scaled.shape[1]
    
    circuit = create_quantum_circuit(n_qubits, n_features)
    
    embedded_features = []
    for sample in X_scaled:
        result = circuit(sample)
        embedded_features.append(result)
    
    return np.array(embedded_features)

# Classification and metrics
def run_classification(X_embedded, y_encoded):
    X_train, X_test, y_train, y_test = train_test_split(X_embedded, y_encoded, test_size=0.2, random_state=42)
    
    classifiers = {
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42)
    }
    
    results = []
    
    for name, clf in classifiers.items():
        fit_start = time.time()
        clf.fit(X_train, y_train)
        fit_time = time.time() - fit_start
        
        predict_start = time.time()
        y_pred = clf.predict(X_test)
        predict_time = time.time() - predict_start
        
        metrics = {
            'Classifier': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1': f1_score(y_test, y_pred),
            'ROC AUC': roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]),
            'Kappa': cohen_kappa_score(y_test, y_pred),
            'Fit Time': fit_time,
            'Predict Time': predict_time
        }
        results.append(metrics)
    
    return pd.DataFrame(results)

def main():
    # Load and prepare data
    print("Loading and preparing data...")
    X_scaled, y_encoded = prepare_data('data-30s.csv')
    
    # Quantum embedding
    print("Performing quantum embedding...")
    X_embedded = quantum_embedding(X_scaled)
    
    # Classification and metrics
    print("Running classifications...")
    results_df = run_classification(X_embedded, y_encoded)
    
    # Display results
    print("\nClassification Metrics Comparison:")
    print(tabulate(results_df.round(4), headers='keys', tablefmt='pretty', showindex=False))
    
    # Save results
    #results_df.to_csv('quantum_classification_results.csv', index=False)
    print("\nResults saved to 'quantum_classification_results.csv'")

if __name__ == "__main__":
    main()


Loading and preparing data...
Performing quantum embedding...
Running classifications...

Classification Metrics Comparison:
+---------------+----------+-----------+--------+--------+---------+--------+----------+--------------+
|  Classifier   | Accuracy | Precision | Recall |   F1   | ROC AUC | Kappa  | Fit Time | Predict Time |
+---------------+----------+-----------+--------+--------+---------+--------+----------+--------------+
| Decision Tree |  0.7311  |  0.7243   | 0.7776 |  0.75  | 0.7379  |  0.46  |  0.0507  |    0.001     |
| Random Forest |  0.7434  |  0.7324   | 0.7961 | 0.7629 | 0.8399  | 0.4844 |  2.0463  |    0.0556    |
+---------------+----------+-----------+--------+--------+---------+--------+----------+--------------+

Results saved to 'quantum_classification_results.csv'


In [20]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
from tabulate import tabulate
from tqdm import tqdm
import time

def prepare_data(filepath):
    df = pd.read_csv(filepath)
    X = df.iloc[:, :23]
    y = df.iloc[:, 23]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_scaled = scaler.fit_transform(X)
    
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    return X_scaled, y_encoded

def create_quantum_circuit(n_qubits, n_features, pca_dim):
    dev = qml.device("default.qubit", wires=n_qubits)
    
    @qml.qnode(dev)
    def circuit(inputs):
        # Basis embedding
        for i in range(n_features):
            qml.RX(inputs[i], wires=i % n_qubits)
        
        # Entanglement layer
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])
            
        # Measure enough observables for desired PCA dimension
        return [qml.expval(qml.PauliZ(i)) for i in range(pca_dim)]
    
    return circuit

def quantum_embedding(X_scaled, pca_dim):
    n_qubits = max(pca_dim, 4)  # Ensure enough qubits for measurements
    n_samples = X_scaled.shape[0]
    n_features = X_scaled.shape[1]
    
    circuit = create_quantum_circuit(n_qubits, n_features, pca_dim)
    
    embedded_features = []
    for sample in tqdm(X_scaled, desc=f"Quantum Embedding (PCA={pca_dim})"):
        result = circuit(sample)
        embedded_features.append(result)
    
    return np.array(embedded_features)

def run_classification(X_embedded, y_encoded, pca_dim):
    X_train, X_test, y_train, y_test = train_test_split(X_embedded, y_encoded, test_size=0.2, random_state=42)
    
    classifiers = {
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42)
    }
    
    results = []
    
    for name, clf in classifiers.items():
        print(f"\nTraining {name} for PCA={pca_dim}...")
        fit_start = time.time()
        clf.fit(X_train, y_train)
        fit_time = time.time() - fit_start
        
        predict_start = time.time()
        y_pred = clf.predict(X_test)
        predict_time = time.time() - predict_start
        
        metrics = {
            'PCA Dim': pca_dim,
            'Classifier': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1': f1_score(y_test, y_pred),
            'ROC AUC': roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]),
            'Kappa': cohen_kappa_score(y_test, y_pred),
            'Fit Time': fit_time,
            'Predict Time': predict_time
        }
        results.append(metrics)
    
    return results

def run_classical_classification(X_scaled, y_encoded):
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)
    
    classifiers = {
        'Decision Tree': DecisionTreeClassifier(random_state=42),
        'Random Forest': RandomForestClassifier(random_state=42)
    }
    
    results = []
    
    for name, clf in classifiers.items():
        print(f"\nTraining Classical {name}...")
        fit_start = time.time()
        clf.fit(X_train, y_train)
        fit_time = time.time() - fit_start
        
        predict_start = time.time()
        y_pred = clf.predict(X_test)
        predict_time = time.time() - predict_start
        
        metrics = {
            'PCA Dim': 'Classical',
            'Classifier': f'Classical {name}',
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1': f1_score(y_test, y_pred),
            'ROC AUC': roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]),
            'Kappa': cohen_kappa_score(y_test, y_pred),
            'Fit Time': fit_time,
            'Predict Time': predict_time
        }
        results.append(metrics)
    
    return results


In [21]:
def main():
    print("Loading and preparing data...")
    X_scaled, y_encoded = prepare_data('data-30s.csv')
    
    # Start with classical classification
    print("\nRunning Classical Classification...")
    all_results = run_classical_classification(X_scaled, y_encoded)
    
    pca_dimensions = [2, 10, 15]
    
    # Then do quantum embeddings
    for pca_dim in pca_dimensions:
        print(f"\nProcessing PCA dimension = {pca_dim}")
        X_embedded = quantum_embedding(X_scaled, pca_dim)
        results = run_classification(X_embedded, y_encoded, pca_dim)
        all_results.extend(results)
    
    # Create and display results table
    results_df = pd.DataFrame(all_results)
    print("\nClassification Metrics Comparison (Classical vs Quantum):")
    print(tabulate(results_df.round(4), headers='keys', tablefmt='pretty', showindex=False))
    
    # Save results
    #results_df.to_csv('quantum_classical_comparison_results.csv', index=False)
    print("\nResults saved to 'quantum_classical_comparison_results.csv'")
    
if __name__ == "__main__":
    main()

Loading and preparing data...

Running Classical Classification...

Training Classical Decision Tree...

Training Classical Random Forest...

Processing PCA dimension = 2


Quantum Embedding (PCA=2): 100%|████████████████████████████████████████████████| 14651/14651 [02:12<00:00, 110.59it/s]



Training Decision Tree for PCA=2...

Training Random Forest for PCA=2...

Processing PCA dimension = 10


Quantum Embedding (PCA=10): 100%|████████████████████████████████████████████████| 14651/14651 [02:52<00:00, 84.85it/s]



Training Decision Tree for PCA=10...

Training Random Forest for PCA=10...

Processing PCA dimension = 15


Quantum Embedding (PCA=15): 100%|████████████████████████████████████████████████| 14651/14651 [09:39<00:00, 25.30it/s]



Training Decision Tree for PCA=15...

Training Random Forest for PCA=15...

Classification Metrics Comparison (Classical vs Quantum):
+-----------+-------------------------+----------+-----------+--------+--------+---------+--------+----------+--------------+
|  PCA Dim  |       Classifier        | Accuracy | Precision | Recall |   F1   | ROC AUC | Kappa  | Fit Time | Predict Time |
+-----------+-------------------------+----------+-----------+--------+--------+---------+--------+----------+--------------+
| Classical | Classical Decision Tree |  0.8646  |  0.8494   | 0.898  | 0.873  | 0.8782  | 0.7281 |  0.309   |     0.0      |
| Classical | Classical Random Forest |  0.9017  |  0.8784   | 0.9408 | 0.9085 | 0.9663  | 0.8027 |  3.8283  |    0.0518    |
|     2     |      Decision Tree      |  0.7311  |  0.7243   | 0.7776 |  0.75  | 0.7379  |  0.46  |  0.0531  |     0.0      |
|     2     |      Random Forest      |  0.7434  |  0.7324   | 0.7961 | 0.7629 | 0.8399  | 0.4844 |  2.1142  

In [27]:
X_scaled, y_encoded = prepare_data('data-30s.csv')

In [30]:
X_scaled.shape[1]

23

In [33]:
import pennylane as qml
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
from sklearn.decomposition import PCA
from tabulate import tabulate
from tqdm import tqdm
import time

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, cohen_kappa_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from tqdm import tqdm
import time
from tabulate import tabulate


def prepare_data(filepath):
    df = pd.read_csv(filepath)
    X = df.iloc[:, :23]
    y = df.iloc[:, 23]
    
    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_scaled = scaler.fit_transform(X)
    
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    
    return X_scaled, y_encoded

def create_quantum_circuit(n_qubits, n_features, pca_dim):
    dev = qml.device("default.qubit", wires=n_qubits)
    
    @qml.qnode(dev)
    def circuit(inputs):
        for i in range(n_features):
            qml.RX(inputs[i], wires=i % n_qubits)
        
        for i in range(n_qubits-1):
            qml.CNOT(wires=[i, i+1])
            
        return [qml.expval(qml.PauliZ(i)) for i in range(pca_dim)]
    
    return circuit

def quantum_embedding(X_scaled, pca_dim):
    n_qubits = max(pca_dim, 2)
    n_samples = X_scaled.shape[0]
    n_features = X_scaled.shape[1]
    
    circuit = create_quantum_circuit(n_qubits, n_features, pca_dim)
    
    embedded_features = []
    for sample in tqdm(X_scaled, desc=f"Quantum Embedding (PCA={pca_dim})"):
        result = circuit(sample)
        embedded_features.append(result)
    
    return np.array(embedded_features)

def classical_pca(X_scaled, n_components):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(X_scaled)

def run_classification(X, y_encoded, method, pca_dim):
    X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)
    
    classifiers = {
        'Random Forest': RandomForestClassifier(random_state=42),
        'CatBoost': CatBoostClassifier(random_state=42, verbose=False),
        'AdaBoost': AdaBoostClassifier(random_state=42),
        'Extra Trees': ExtraTreesClassifier(random_state=42),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42),
        'XGBoost': XGBClassifier(random_state=42),
        'Logistic Regression': LogisticRegression(random_state=42),
        'KNN': KNeighborsClassifier(),
        'Decision Tree': DecisionTreeClassifier(random_state=42)
    }
    
    results = []
    
    for name, clf in classifiers.items():
        print(f"\nTraining {method} {name} for PCA={pca_dim}...")
        fit_start = time.time()
        clf.fit(X_train, y_train)
        fit_time = time.time() - fit_start
        
        predict_start = time.time()
        y_pred = clf.predict(X_test)
        predict_time = time.time() - predict_start
        
        metrics = {
            'Method': method,
            'PCA Dim': pca_dim,
            'Classifier': name,
            'Accuracy': accuracy_score(y_test, y_pred),
            'Precision': precision_score(y_test, y_pred),
            'Recall': recall_score(y_test, y_pred),
            'F1': f1_score(y_test, y_pred),
            'ROC AUC': roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1]),
            'Kappa': cohen_kappa_score(y_test, y_pred),
            'Fit Time': fit_time,
            'Predict Time': predict_time
        }
        results.append(metrics)
    
    return results



In [34]:
def main():
    print("Loading and preparing data...")
    X_scaled, y_encoded = prepare_data('data-30s.csv')
    
    pca_dimensions = [2,10,15]
    all_results = []
    
    # Classical PCA
    for pca_dim in pca_dimensions:
        print(f"\nProcessing Classical PCA dimension = {pca_dim}")
        X_classical_pca = classical_pca(X_scaled, pca_dim)
        results = run_classification(X_classical_pca, y_encoded, "Classical PCA", pca_dim)
        all_results.extend(results)
    
    # Quantum Embeddings
    for pca_dim in pca_dimensions:
        print(f"\nProcessing Quantum PCA dimension = {pca_dim}")
        X_quantum = quantum_embedding(X_scaled, pca_dim)
        results = run_classification(X_quantum, y_encoded, "Quantum", pca_dim)
        all_results.extend(results)
    
    # Create and display results table
    results_df = pd.DataFrame(all_results)
    print("\nClassification Metrics Comparison (Classical vs Quantum):")
    print(tabulate(results_df.round(4), headers='keys', tablefmt='pretty', showindex=False))
    
    # Save results
    results_df.to_csv('comprehensive_comparison_results.csv', index=False)
    print("\nResults saved to 'comprehensive_comparison_results.csv'")

if __name__ == "__main__":
    main()


Loading and preparing data...

Processing Classical PCA dimension = 2

Training Classical PCA Random Forest for PCA=2...

Training Classical PCA CatBoost for PCA=2...

Training Classical PCA AdaBoost for PCA=2...

Training Classical PCA Extra Trees for PCA=2...

Training Classical PCA Gradient Boosting for PCA=2...

Training Classical PCA XGBoost for PCA=2...

Processing Quantum PCA dimension = 2


Quantum Embedding (PCA=2): 100%|████████████████████████████████████████████████| 14651/14651 [02:13<00:00, 109.79it/s]



Training Quantum Random Forest for PCA=2...

Training Quantum CatBoost for PCA=2...

Training Quantum AdaBoost for PCA=2...

Training Quantum Extra Trees for PCA=2...

Training Quantum Gradient Boosting for PCA=2...

Training Quantum XGBoost for PCA=2...

Classification Metrics Comparison (Classical vs Quantum):
+---------------+---------+-------------------+----------+-----------+--------+--------+---------+--------+----------+--------------+
|    Method     | PCA Dim |    Classifier     | Accuracy | Precision | Recall |   F1   | ROC AUC | Kappa  | Fit Time | Predict Time |
+---------------+---------+-------------------+----------+-----------+--------+--------+---------+--------+----------+--------------+
| Classical PCA |    2    |   Random Forest   |  0.7731  |  0.7599   | 0.8224 | 0.7899 | 0.8684  | 0.5442 |  2.2681  |    0.0665    |
| Classical PCA |    2    |     CatBoost      |  0.7489  |  0.7344   | 0.8079 | 0.7694 | 0.8431  | 0.4951 |  9.7423  |    0.0162    |
| Classical PCA

In [37]:
df1=pd.read_csv('data-30s.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14651 entries, 0 to 14650
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   duration            14651 non-null  float64
 1   total_fiat          14651 non-null  float64
 2   total_biat          14651 non-null  float64
 3   min_fiat            14651 non-null  float64
 4   min_biat            14651 non-null  float64
 5   max_fiat            14651 non-null  float64
 6   max_biat            14651 non-null  float64
 7   mean_fiat           14651 non-null  float64
 8   mean_biat           14651 non-null  float64
 9   flowPktsPerSecond   14651 non-null  float64
 10  flowBytesPerSecond  14651 non-null  float64
 11  min_flowiat         14651 non-null  float64
 12  max_flowiat         14651 non-null  float64
 13  mean_flowiat        14651 non-null  float64
 14  std_flowiat         14651 non-null  float64
 15  min_active          14651 non-null  float64
 16  mean